In [109]:
from typing import Tuple
import pandas as pd
import sympy as sp
import sys
import os
import json

sys.path.append(os.path.dirname(os.getcwd()))

from lib.sympy_helper import normalise

In [110]:
reference_csv = f"../outputs/max-corr-5/hall_of_fame.csv"
max_reference_complexity = None
correction_csv = f"../outputs/msre-ref-4/hall_of_fame.csv"
max_correction_complexity = None

In [111]:
d, t, ρ, σ = sp.symbols('d, t, ρ, σ')
variable_replacement_map = {
    "x1": d,
    "x2": t,
    "x3": ρ,
    "x4": σ
}

In [112]:
reference_equations = pd.read_csv(correction_csv)
if max_reference_complexity == None:
    reference_equation = reference_equations.iloc[-1]['Equation']
else:
    mask = reference_equations["Complexity"] <= max_reference_complexity
    idx = reference_equations.loc[mask, "Complexity"].idxmax()
    reference_equation = reference_equations.loc[idx]['Equation']
sympy_reference_unnormalized = sp.sympify(reference_equation)
sympy_reference = normalise(sympy_reference_unnormalized)


correction_equations = pd.read_csv(reference_csv)
if max_correction_complexity == None:
    correction_equation = correction_equations.iloc[-1]['Equation']
else:
    mask = correction_equations["Complexity"] <= max_correction_complexity
    idx = correction_equations.loc[mask, "Complexity"].idxmax()
    correction_equation = correction_equations.loc[idx]['Equation']
sympy_correction_unnormalized = sp.sympify(correction_equation)
sympy_correction = normalise(sympy_correction_unnormalized)

In [113]:
def get_constant_equation(eq: sp.Expr) -> Tuple[dict, str]:
    def _new_const(counter):
        name = f'c{counter}'
        return sp.Symbol(name)
    eq = eq.subs(variable_replacement_map)

    replacements = {}
    default_values = []
    constants = []
    counter = 0
    for node in sp.preorder_traversal(eq):    
        if isinstance(node, sp.Number) and node not in replacements:
            c = _new_const(counter)
            counter += 1
            replacements[node] = c
            default_values.append(node)
            constants.append(c)

    expr_with_constants = eq.xreplace(replacements)

    constant_equation = sp.latex(expr_with_constants)
    flipped_dict = {str(value): str(key) for key, value in replacements.items()}
    return flipped_dict, constant_equation

In [114]:
ref_replacements, ref_latex = get_constant_equation(sympy_reference)
corr_replacements, corr_latex = get_constant_equation(sympy_correction)

In [115]:
corr_latex

'\\left(c_{0} t \\left(c_{1} + c_{11} \\left(c_{10} d t \\left|{c_{6} + c_{7} d^{c_{8}} + c_{9} t}\\right|^{c_{0}} + c_{4} d^{c_{5}} \\left|{c_{6} + c_{7} d^{c_{8}} + c_{9} t}\\right|^{c_{0}} + t\\right)^{c_{0}} + c_{12} t \\left(c_{10} d t \\left|{c_{6} + c_{7} d^{c_{8}} + c_{9} t}\\right|^{c_{0}} + c_{4} d^{c_{5}} \\left|{c_{6} + c_{7} d^{c_{8}} + c_{9} t}\\right|^{c_{0}} + t\\right)^{c_{0}} + c_{13} t \\left(c_{10} d t ρ \\left|{c_{6} + c_{7} d^{c_{8}} + c_{9} t}\\right|^{c_{0}} + c_{4} d^{c_{5}} ρ \\left|{c_{6} + c_{7} d^{c_{8}} + c_{9} t}\\right|^{c_{0}} + t ρ\\right)^{c_{0}} + c_{14} ρ \\left(c_{10} d t \\left|{c_{6} + c_{7} d^{c_{8}} + c_{9} t}\\right|^{c_{0}} + c_{4} d^{c_{5}} \\left|{c_{6} + c_{7} d^{c_{8}} + c_{9} t}\\right|^{c_{0}} + t\\right)^{c_{0}} + c_{2} t + c_{3} \\left(c_{10} d t ρ \\left|{c_{6} + c_{7} d^{c_{8}} + c_{9} t}\\right|^{c_{0}} + c_{4} d^{c_{5}} ρ \\left|{c_{6} + c_{7} d^{c_{8}} + c_{9} t}\\right|^{c_{0}} + t ρ\\right)^{c_{0}}\\right)^{c_{0}}\\right)^{c_{1

In [116]:
prefix = "reference"
if max_reference_complexity:
    prefix += f"_{max_reference_complexity}"
with open(f'../{prefix}_constants.json', 'w') as json_file:
    json.dump(ref_replacements, json_file, indent=4)
with open(f'../{prefix}_latex.txt', 'w') as f:
    f.write(ref_latex)

In [117]:
prefix = "correction"
if max_reference_complexity:
    prefix += f"_{max_reference_complexity}"
with open(f'../{prefix}_constants.json', 'w') as json_file:
    json.dump(corr_replacements, json_file, indent=4)
with open(f'../{prefix}_latex.txt', 'w', encoding="utf-8") as f:
    f.write(corr_latex)